In [1]:
import numpy as np
import pandas as pd

In [6]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric, add_changepoints_to_plot, plot_plotly
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## Read in the data from Kaggle

In [139]:
df = pd.read_csv('2019_nCoV_data.csv')
# conf_df = pd.read_csv('time_series_2019_ncov_confirmed.csv')
# deaths_df = pd.read_csv('time_series_2019_ncov_deaths.csv')
# recv_df = pd.read_csv('time_series_2019_ncov_recovered.csv')

conf_df = pd.read_csv('time_series_2019-ncov-Confirmed.csv')
deaths_df = pd.read_csv('time_series_2019-ncov-Deaths.csv')
recv_df = pd.read_csv('time_series_2019-ncov-Recovered.csv')

In [140]:
df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020 12:00:00,Anhui,China,01/22/2020 12:00:00,1.0,0.0,0.0
1,2,01/22/2020 12:00:00,Beijing,China,01/22/2020 12:00:00,14.0,0.0,0.0
2,3,01/22/2020 12:00:00,Chongqing,China,01/22/2020 12:00:00,6.0,0.0,0.0
3,4,01/22/2020 12:00:00,Fujian,China,01/22/2020 12:00:00,1.0,0.0,0.0
4,5,01/22/2020 12:00:00,Gansu,China,01/22/2020 12:00:00,0.0,0.0,0.0


In [141]:
dates = list(conf_df.columns[4:])
dates1 = list(recv_df.columns[4:])

### We would want to aggregate the confirmed/death/recovered data under one date frame

In [142]:
conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], 
                       axis=1, sort=False)

In [144]:
full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table['Recovered'] = full_table['Recovered'].astype('int')
full_table.dtypes

Province/State            object
Country/Region            object
Lat                      float64
Long                     float64
Date              datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
dtype: object

In [145]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.82571,117.2264,2020-01-22,1,0,0
1,Beijing,Mainland China,40.18238,116.4142,2020-01-22,14,0,0
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-22,6,0,0
3,Fujian,Mainland China,26.07783,117.9895,2020-01-22,1,0,0
4,Gansu,Mainland China,36.06110,103.8343,2020-01-22,0,0,0


In [146]:
full_table['Country/Region'].unique()

array(['Mainland China', 'Thailand', 'Japan', 'South Korea', 'Taiwan',
       'US', 'Macau', 'Hong Kong', 'Singapore', 'Vietnam', 'France',
       'Nepal', 'Malaysia', 'Canada', 'Australia', 'Cambodia',
       'Sri Lanka', 'Germany', 'Finland', 'United Arab Emirates',
       'Philippines', 'India', 'Italy', 'UK', 'Russia', 'Sweden', 'Spain',
       'Belgium', 'Others', 'Egypt', 'Iran'], dtype=object)

## Add a column for population of the city

In [147]:
full_table['Province/State'].unique()

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin',
       'Liaoning', 'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong',
       'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Xinjiang',
       'Yunnan', 'Zhejiang', nan, 'Taiwan', 'Seattle, WA', 'Chicago, IL',
       'Tempe, AZ', 'Macau', 'Hong Kong', 'Toronto, ON',
       'British Columbia', 'Orange, CA', 'Los Angeles, CA',
       'New South Wales', 'Victoria', 'Queensland', 'London, ON',
       'Santa Clara, CA', 'South Australia', 'Boston, MA',
       'San Benito, CA', 'Madison, WI', 'Diamond Princess cruise ship',
       'San Diego County, CA', 'San Antonio, TX'], dtype=object)

In [151]:
def label_race (row):
    if row['Province/State'] == 'Anhui' :
        return 62.0
    if row['Province/State'] == 'Beijing' :
        return 21.54
    if row['Province/State'] == 'Chongqing':
        return 30.48
    if row['Province/State'] == 'Fujian':
        return 38.56
    if row['Province/State']  == 'Gansu':
        return 25.58
    if row['Province/State']  == 'Guangdong':
        return 113.46 
    if row['Province/State']  == 'Guangxi':
        return 48.38 
    if row['Province/State']  == 'Guizhou':
        return 34.75 
    if row['Province/State']  == 'Hainan':
        return 9.3
    if row['Province/State']  == 'Hebei':
        return 74.7
    if row['Province/State']  == 'Heilongjiang':
        return 38.31
    if row['Province/State']  == 'Henan':
        return 94 
    if row['Province/State']  == 'Hubei':
        return 58.5
    if row['Province/State']  == 'Hunan':
        return 67.37
    if row['Province/State']  == 'Inner Mongolia':
        return 24.71
    if row['Province/State']  == 'Jiangsu':
        return 80.4
    if row['Province/State']  == 'Jiangxi':
        return 45.2
    if row['Province/State']  == 'Jilin':
        return 27.46
    if row['Province/State']  == 'Liaoning':
        return 43.9 
    if row['Province/State']  == 'Ningxia':
        return 6.3
    if row['Province/State']  == 'Qinghai':
        return 5.6 
    if row['Province/State']  == 'Shaanxi':
        return 37.33
    if row['Province/State']  == 'Shandong':
        return 90.0
    if row['Province/State']  == 'Shanghai':
        return 24.24
    if row['Province/State']  == 'Shanxi':
        return 36.5
    if row['Province/State']  == 'Sichuan':
        return 81.1
    if row['Province/State']  == 'Tianjin':
        return 15.0 
    if row['Province/State']  == 'Tibet':
        return 3.2
    if row['Province/State']  == 'Xinjiang':
        return 22.0
    if row['Province/State']  == 'Yunnan':
        return 46.0
    if row['Province/State']  == 'Zhejiang':
        return 57.4
    if row['Province/State']  == 'Taiwan':
        return 23.78
    if row['Province/State']  == 'Seattle, WA':
        return 0.72
    if row['Province/State']  == 'Chicago, IL':
        return 2.7
    if row['Province/State']  == 'Tempe, AZ':
        return 0.185
    if row['Province/State']  == 'Macau':
        return 0.62
    if row['Province/State']  == 'Hong Kong':
        return 7.4
    if row['Province/State']  == 'Toronto, ON':
        return 2.93
    if row['Province/State']  == 'British Columbia':
        return 5.07
    if row['Province/State']  == 'Orange, CA':
        return 0.14
    if row['Province/State']  == 'Los Angeles, CA':
        return 4.0
    if row['Province/State']  == 'New South Wales':
        return 7.54
    if row['Province/State']  == 'Victoria':
        return 6.4
    if row['Province/State']  == 'Queensland':
        return 5.07
    if row['Province/State']  == 'London, ON':
        return 0.4
    if row['Province/State']  == 'Santa Clara, CA':
        return 0.127
    if row['Province/State']  == 'South Australia':
        return 1.677
    if row['Province/State']  == 'Boston, MA':
        return 0.685
    if row['Province/State']  == 'San Benito, CA':
        return 0.06
    if row['Province/State']  == 'Madison, WI':
        return 0.255
    if row['Province/State']  == 'Diamond Princess cruise ship':
        return 4/1000
    
    # Below are countries without going to specific cities
    if row['Country/Region']  == 'Thailand':
        return 69.04
    if row['Country/Region']  == 'Japan':
        return 126.8
    if row['Country/Region']  == 'South Korea':
        return 51.47
    if row['Country/Region']  == 'Singapore':
        return 5.6
    if row['Country/Region']  == 'Vietnam':
        return 95.54
    if row['Country/Region']  == 'France':
        return 67.0
    if row['Country/Region']  == 'Nepal':
        return 29.3
    if row['Country/Region']  == 'Malaysia':
        return 31.62
    if row['Country/Region']  == 'Cambodia':
        return 16.01
    if row['Country/Region']  == 'Sri Lanka':
        return 21.44
    if row['Country/Region']  == 'Germany':
        return 82.79
    if row['Country/Region']  == 'Finland':
        return 5.5
    if row['Country/Region']  == 'United Arab Emirates':
        return 9.4
    if row['Country/Region']  == 'Philippines':
        return 104.9
    if row['Country/Region']  == 'India':
        return 1339
    if row['Country/Region']  == 'Italy':
        return 60.48
    if row['Country/Region']  == 'UK':
        return 66.44
    if row['Country/Region']  == 'Russia':
        return 144.5
    if row['Country/Region']  == 'Sweden':
        return 10.12
    if row['Country/Region']  == 'Spain':
        return 46.66
    if row['Country/Region']  == 'Belgium':
        return 11.4
        
    return 10

In [152]:
full_table.loc[full_table['Province/State'].isna()]['Country/Region'].unique()

array([], dtype=object)

## Create a new column for the population of the country

In [153]:
full_table['population'] = full_table.apply (lambda row: label_race(row), axis=1)

In [154]:
# filling missing values with 0 in columns ('Confirmed', 'Deaths', 'Recovered')
full_table[['Confirmed', 'Deaths', 'Recovered']] = full_table[['Confirmed', 'Deaths', 'Recovered']].fillna(0)
full_table[['Province/State']] = full_table[['Province/State']].fillna('NA')

# cases in the Diamond Princess cruise ship
ship = full_table[full_table['Province/State']=='Diamond Princess cruise ship']

# full table
full_table = full_table[full_table['Province/State']!='Diamond Princess cruise ship']



full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,population
0,Anhui,Mainland China,31.82571,117.2264,2020-01-22,1,0,0,62.00
1,Beijing,Mainland China,40.18238,116.4142,2020-01-22,14,0,0,21.54
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-22,6,0,0,30.48
3,Fujian,Mainland China,26.07783,117.9895,2020-01-22,1,0,0,38.56
4,Gansu,Mainland China,36.06110,103.8343,2020-01-22,0,0,0,25.58


## Severity computed by No.Confirmed / Population

In [155]:
# derived dataframes
china = full_table[full_table['Country/Region']=='Mainland China']
row = full_table[full_table['Country/Region']!='Mainland China']

full_latest = full_table[full_table['Date'] == max(full_table['Date'])].reset_index()
china_latest = full_latest[full_latest['Country/Region']=='Mainland China']
row_latest = full_latest[full_latest['Country/Region']!='Mainland China']

full_latest_grouped = full_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered','population'].sum().reset_index()
china_latest_grouped = china_latest.groupby('Province/State')['Confirmed', 'Deaths', 'Recovered','population'].sum().reset_index()
row_latest_grouped = row_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered','population'].sum().reset_index()

In [202]:
full_latest_grouped['Severity'] = full_latest_grouped['Confirmed']/full_latest_grouped['population']

In [200]:
fig = px.choropleth(china_latest_grouped, locations='Province/State', locationmode='country names', color='Severity',
                    hover_name='Province/State',color_continuous_scale="Viridis", title='Severity in China'
                   )
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [175]:
temp = full_latest.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered','population'].max()
temp['severity'] = temp.Confirmed/temp.population
temp.drop(columns='population', inplace = True)
temp.style.background_gradient(cmap='Pastel1_r')

In [191]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,population
0,Anhui,Mainland China,31.82571,117.2264,2020-01-22,1,0,0,62.00
1,Beijing,Mainland China,40.18238,116.4142,2020-01-22,14,0,0,21.54
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-22,6,0,0,30.48
3,Fujian,Mainland China,26.07783,117.9895,2020-01-22,1,0,0,38.56
4,Gansu,Mainland China,36.06110,103.8343,2020-01-22,0,0,0,25.58


In [193]:
fig = px.treemap(china_latest.sort_values(by='Confirmed', ascending=False).reset_index(drop=True), 
           path=["Province/State"], values="Confirmed", title='Number of Confirmed Cases in Chinese Provinces')
fig.show()

fig = px.treemap(china_latest.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
           path=["Province/State"], values="Deaths", title='Number of Deaths Reported in Chinese Provinces')
fig.show()

In [194]:
fig = px.treemap(row_latest, path=["Country/Region"], 
                 values="Confirmed", title='Number of Confirmed Cases outside china')
fig.show()

fig = px.treemap(row_latest, path=["Country/Region"], 
                 values="Deaths", title='Number of Deaths outside china')
fig.show()

In [190]:
formated_gdf = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf = formated_gdf[formated_gdf['Country/Region']!='Mainland China']
formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')

fig = px.scatter_geo(formated_gdf[formated_gdf['Country/Region']!='Mainland China'], 
                     locations="Country/Region", locationmode='country names', 
                     color="Confirmed", size='Confirmed', hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf['Confirmed'])+2], 
                     projection="natural earth", animation_frame="Date", 
                     title='Spread outside China over time')
fig.update(layout_coloraxis_showscale=False)
fig.show()

# -----------------------------------------------------------------------------------

china_map = china.groupby(['Date', 'Province/State'])['Confirmed', 'Deaths', 'Recovered', 
                                                      'Lat', 'Long'].max()
china_map = china_map.reset_index()
china_map['size'] = china_map['Confirmed'].pow(0.5)
china_map['Date'] = pd.to_datetime(china_map['Date'])
china_map['Date'] = china_map['Date'].dt.strftime('%m/%d/%Y')
china_map.head()

fig = px.scatter_geo(china_map, lat='Lat', lon='Long', scope='asia',
                     color="size", size='size', hover_name='Province/State', 
                     hover_data=['Confirmed', 'Deaths', 'Recovered'],
                     projection="natural earth", animation_frame="Date", 
                     title='Spread in China over time')
fig.update(layout_coloraxis_showscale=False)
fig.show()